In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dataset

In [ ]:
from pathlib import Path

class config:
    IMG_PATH = "/kaggle/input/the-car-connection-picture-dataset"
    ORIGINAL_PROJECT_PATH ="/kaggle/input/train-dataset/train"
    ORIGINAL_PROJECT_LABELS = "/kaggle/input/train-dataset/train.csv"

Loading the data in a dataset

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torchvision import transforms
from torch.utils.data import Dataset

class CarDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.transform_img = transforms.Compose([transforms.ToTensor()])
        self.files = []
        self._get_files()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = Path(self.img_dir) / Path(self.files[idx])
        image = read_image(img_path.as_posix())
        if self.transform:
            image = self.transform(image)
        return image
    
    def _get_files(self):
        p = Path(self.img_dir).glob('**/*')
        self.files = sorted([x.name for x in p if x.is_file()])

        
cars_dataset = CarDataset(config.IMG_PATH)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(np.transpose(cars_dataset[0],(1,2,0)))

### Checking the dataset

1. The size of all the images

In [ ]:
cars_dataset.files[count]

In [ ]:
import pandas as pd
from tqdm import tqdm

shapes = []

# See the size of all the pictures
for count, im in tqdm(enumerate(cars_dataset)):
    rgb, h, w = im.shape
    shapes.append([cars_dataset.files[count],rgb, h , w])
    
shapes_df = pd.DataFrame(shapes,columns = ["path","RGB","h","w"])
shapes_df.head()

In [ ]:
print("Unique values for the Heights:",shapes_df["h"].unique())
print("Unique values for the Width:",shapes_df["w"].unique())

Widths are unique and not the heights

In [ ]:
print("Heights:", shapes_df["h"].value_counts())

fig, ax = plt.subplots()
labels = shapes_df["h"].value_counts().index
values = (shapes_df["h"].value_counts() / len(shapes_df["h"])) *100
ax.bar(labels, values,20)
ax.set_xlabel("Heigths")
ax.set_ylabel("Percentage")
plt.title("Percentage of Heights")

**14000 data is enought for training a model, we will only keep the 213x320 datas**

In [ ]:
final_df = shapes_df["path"].loc[shapes_df["h"] == 213]
final_df

### New dataset

In [ ]:
cars_dataset.files = final_df.values

In [ ]:
len(cars_dataset)

# Second part: labeling non car data

1. Importing the images

In [ ]:
df_original_dataset_labels["bounding_boxes"].isna().sum()

In [ ]:
df_original_dataset_labels = pd.read_csv(config.ORIGINAL_PROJECT_LABELS)
empty_images = df_original_dataset_labels["bounding_boxes"].isna().sum()
print("Number of images without any car:",empty_images)

df_empty_images = df_original_dataset_labels.loc[df_original_dataset_labels["bounding_boxes"].isna()]
print(len(df_empty_images))

# Hog

In [ ]:
#creating hog features
fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True, multichannel=True)